In [2]:
import warnings
warnings.simplefilter('ignore')

from sklearn import svm
import pickle
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import time
import re
import os
import json
import subprocess
import csv
import gzip
from datetime import datetime
import itertools

import matplotlib.pyplot as plt

import sys
sys.path.append('../../../../external_repositories/word2vec-twitter/')

from word2vecReader import Word2Vec

# Load Pretrained w2v Model Vocab
You'll need to have downloaded word2vec_twitter_model.bin into external_repositories/word2vec-twitter from https://www.kaggle.com/hachemsfar/cbigru/version/1. The model is originally created by Fréderic Godin, but the creator's link to the w2v model is no longer active. 

In [3]:
model_path = "../../../../external_repositories/word2vec-twitter/word2vec_twitter_model.bin"
print("Loading the model, this can take some time...")
model = Word2Vec.load_word2vec_format(model_path, binary=True)
print("The vocabulary size is: "+str(len(model.vocab)))

Loading the model, this can take some time...
The vocabulary size is: 3039345


# Get Data

In [4]:
data_file = "../../../../data/russell/processed_hand_labeled_w2v_original.csv.gz"
tweets = pd.read_csv(data_file, dtype=str, compression='gzip')

# Get w2v Features
Here each tweet is its own document.

In [5]:
## version of func borrowed from https://www.kaggle.com/vukglisovic/classification-combining-lda-and-word2vec
def get_w2v_features(w2v_model, words):
    
    index2word_set = set(w2v_model.vocab.keys())  # words known to model
    featureVec = np.zeros(w2v_model.layer1_size, dtype="float32")
    
    ## Initialize a counter for number of words in a review
    nwords = 0
    ## Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    ## Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    
    return featureVec

In [ ]:
%%time
tweets['w2v_features_pretrained'] = list(map(lambda sen_group:
                                      get_w2v_features(model, sen_group),
                                      tweets.text_tokenized))

In [7]:
tweets.to_csv('../../../../data/russell/processed_hand_labeled_w2v_both.csv.gz')